In [1]:
from feature_engineering_utilities import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

2019-12-02-17-46


**Observations with Labels**
---
Using the icd-hcc crosswalk, the icd diagnosis data for each admission and the admissions table, create a labeled dataset with the target variable. We will use this dataframe as the scaffolding to generate all the features that we need and then merge them on. The goal of this ipython notebook is to parse through all the different tables and create a final feature dataset that we can use for our modeling exercise.

In [2]:
icdxw = read_crosswalk()
diagnoses_icd = read_prod_data('diagnoses_icd')
admissions = read_prod_data('admissions')
df = add_aki_hcc_label(diagnoses_icd, icdxw, admissions)
hcc_data = create_hcc_labeled_dataset(diagnoses_icd, icdxw)

printer('labeled dataset')
df.shape
df.sample(5)
df.hcc_cd_135.value_counts(normalize=True)
del diagnoses_icd
del icdxw

diagnoses_icd (651047, 5)
admissions (58976, 19)
labeled dataset
-------------------


(58976, 5)

,hadm_id,subject_id,hcc_cd_135,admittime,dischtime
24085,140772,8378,0,2177-01-24 13:22:00,2177-01-29 14:50:00
12322,120779,3214,0,2164-07-25 08:00:00,2164-08-02 17:10:00
54871,193002,25708,0,2178-11-19 00:48:00,2178-11-24 19:48:00
39560,167207,94172,0,2193-01-14 07:15:00,2193-01-18 15:06:00
5048,108529,88526,0,2188-07-25 01:46:00,2188-07-31 14:45:00


0    0.806328
1    0.193672
Name: hcc_cd_135, dtype: float64

Notice that 20% of the data are positives. We might want to consider undersampling the negatives or oversampling the positives (when we model the data) so that we can have an even distribution of this event.

**Demographic Features**
---
Read in the patients table and create demographic features. Some of the main risk factors for "Acute Kidney Injury" (AKI) are **age**, **gender** and **ethnicity**. We also pull out the **admission type** of the stay to see if knowing that the hospital admission was due to an emergency condition or an elective procedure could be useful to us.

In [3]:
patients = read_prod_data('patients')
demographic_features = create_demographics_features(admissions, patients)
del patients
printer('demographic features')
demographic_features.shape
demographic_features.columns
demographic_features.sample(5)

patients (46520, 8)
demographic features
-------------------


(58976, 48)

Index(['hadm_id', 'ft_age', 'ft_gender', 'ft_admit_type_elective',
       'ft_admit_type_emergency', 'ft_admit_type_newborn',
       'ft_admit_type_urgent', 'ft_race_american_indian_alaska_native',
       'ft_race_american_indian_alaska_native_federally_recognized_tribe',
       'ft_race_asian', 'ft_race_asian_asian_indian',
       'ft_race_asian_cambodian', 'ft_race_asian_chinese',
       'ft_race_asian_filipino', 'ft_race_asian_japanese',
       'ft_race_asian_korean', 'ft_race_asian_other', 'ft_race_asian_thai',
       'ft_race_asian_vietnamese', 'ft_race_black_african',
       'ft_race_black_african_american', 'ft_race_black_cape_verdean',
       'ft_race_black_haitian', 'ft_race_caribbean_island',
       'ft_race_hispanic_latino_central_american_(other)',
       'ft_race_hispanic_latino_colombian', 'ft_race_hispanic_latino_cuban',
       'ft_race_hispanic_latino_dominican',
       'ft_race_hispanic_latino_guatemalan',
       'ft_race_hispanic_latino_honduran', 'ft_race_hispanic_la

,hadm_id,ft_age,ft_gender,ft_admit_type_elective,ft_admit_type_emergency,ft_admit_type_newborn,ft_admit_type_urgent,ft_race_american_indian_alaska_native,ft_race_american_indian_alaska_native_federally_recognized_tribe,ft_race_asian,ft_race_asian_asian_indian,ft_race_asian_cambodian,ft_race_asian_chinese,ft_race_asian_filipino,ft_race_asian_japanese,ft_race_asian_korean,ft_race_asian_other,ft_race_asian_thai,ft_race_asian_vietnamese,ft_race_black_african,ft_race_black_african_american,ft_race_black_cape_verdean,ft_race_black_haitian,ft_race_caribbean_island,ft_race_hispanic_latino_central_american_(other),ft_race_hispanic_latino_colombian,ft_race_hispanic_latino_cuban,ft_race_hispanic_latino_dominican,ft_race_hispanic_latino_guatemalan,ft_race_hispanic_latino_honduran,ft_race_hispanic_latino_mexican,ft_race_hispanic_latino_puerto_rican,ft_race_hispanic_latino_salvadoran,ft_race_hispanic_or_latino,ft_race_middle_eastern,ft_race_multi_race_ethnicity,ft_race_native_hawaiian_or_other_pacific_islander,ft_race_other,ft_race_patient_declined_to_answer,ft_race_portuguese,ft_race_south_american,ft_race_unable_to_obtain,ft_race_unknown_not_specified,ft_race_white,ft_race_white_brazilian,ft_race_white_eastern_european,ft_race_white_other_european,ft_race_white_russian
9148,115407,21.363450,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6516,110965,55.737166,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
54829,192923,79.676934,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
461,100765,68.109514,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8517,114338,42.910335,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Note that some members don't have an age because they were so old that it might have been too easy to identify them. 
Therefore, their dates of birth were nulled out to protect their identity.

**HCC's for Comorbidities**
---
Create features for HCCs that are comorbidities. The main comoborbidities are as follows:
- diabetes (without complications)
- ckd (stage 5) and (stage 4)
- chf
- vascular disease
- chronic liver disease (which was not calculated)

In [4]:
diabetes_hcc_feature = create_hcc_feature(hcc_data, label='_19', rename_as='hcc_cd_19_dbtes_wo_comp')
ckd5_hcc_feature = create_hcc_feature(hcc_data, label='_136', rename_as='hcc_cd_136_ckd_stg_5')
ckd4_hcc_feature = create_hcc_feature(hcc_data, label='_137', rename_as='hcc_cd_137_ckd_stg_4')
chf_hcc_feature = create_hcc_feature(hcc_data, label='_85', rename_as='hcc_cd_85_chf')
vascular_disease_hcc_feature = create_hcc_feature(hcc_data, label='_108', rename_as='hcc_cd_108_vascular')

del hcc_data
printer('hcc features')
diabetes_hcc_feature.shape
ckd5_hcc_feature.shape
ckd4_hcc_feature.shape
chf_hcc_feature.shape
vascular_disease_hcc_feature.shape
vascular_disease_hcc_feature.sample(5)

hcc features
-------------------


(58976, 2)

(58976, 2)

(58976, 2)

(58976, 2)

(58976, 2)

,hadm_id,hcc_cd_108_vascular
43209,173241,0
41514,170371,0
42345,171800,0
38528,165426,0
18482,131284,0


**Prior Admissions / ICU Stays**
---
One hypothesis is that prior admissions or icu stays for the patient within certain timeframes can be good predictors of acuity/severity of a condition or whether this patient's condition is worsening into AKI.

In [5]:
# admissions, icu stays
icustays = read_prod_data('icustays')
prior_admission_features = create_prior_admissions(admissions, icustays)
del icustays
printer('prior admission/icustays features')
prior_admission_features.shape
prior_admission_features.columns
prior_admission_features.sample(5)

icustays (61532, 12)
prior admission/icustays features
-------------------


(19993, 8)

Index(['hadm_id', 'ft_prior_admission_30', 'ft_prior_admission_60',
       'ft_prior_admission_90', 'ft_prior_admission_120',
       'ft_avg_icu_los_within_30', 'ft_micu_within_30', 'ft_ccu_within_30'],
      dtype='object')

,hadm_id,ft_prior_admission_30,ft_prior_admission_60,ft_prior_admission_90,ft_prior_admission_120,ft_avg_icu_los_within_30,ft_micu_within_30,ft_ccu_within_30
8515,142580,1,1,1,1,5.3577,1,0
11658,158177,1,1,1,1,1.9772,0,0
12334,161663,0,0,0,0,NaN,0,0
19367,196931,1,1,1,1,1.0586,1,0
17325,186671,1,1,1,1,3.1719,0,0


**Prescriptions**
---
Some prescriptions that are used for other diseases can cause AKI and are high risk factors for kidney injury. It would be interesting to know if prescription of those drugs can significantly increase risk in AKI.

In [6]:
# prescription
prescriptions = read_prod_data('prescriptions')
nephrotoxin_features = add_nephrotoxin_features(prescriptions, admissions)
del prescriptions
printer('nephrotoxins features')
nephrotoxin_features.shape
nephrotoxin_features.columns
nephrotoxin_features.sample(5)

prescriptions (4156450, 19)
	 antibiotics
		 bacitracin
		  24
		  48
		  72
		 vancomycin
		  24
		  48
		  72
		 amphotericin
		  24
		  48
		  72
		 cephalosporin
		  24
		  48
		  72
		 aminoglycoloside
		  24
		  48
		  72
		 ciprofloxacin
		  24
		  48
		  72
	 blood_pressure
		 lisinopril
		  24
		  48
		  72
		 ramipril
		  24
		  48
		  72
		 metoprolol
		  24
		  48
		  72
		 candesartan
		  24
		  48
		  72
		 valsartan
		  24
		  48
		  72
		 warfarin
		  24
		  48
		  72
	 edema
		 furosemide
		  24
		  48
		  72
	 nsaid
		 ibuprofen
		  24
		  48
		  72
		 naproxen
		  24
		  48
		  72
		 ketoprofen
		  24
		  48
		  72
	 ulcer
		 cimetidine
		  24
		  48
		  72
	 other
		 propofol
		  24
		  48
		  72
nephrotoxins features
-------------------


(50216, 101)

Index(['hadm_id', 'ft_any_nephrotoxin_rx', 'ft_any_nephrotoxin_rx_within_24',
       'ft_any_nephrotoxin_rx_within_48', 'ft_any_nephrotoxin_rx_within_72',
       'ft_nephrotoxin_bacitracin_rx',
       'ft_nephrotoxin_bacitracin_rx_within_24',
       'ft_nephrotoxin_bacitracin_rx_within_48',
       'ft_nephrotoxin_bacitracin_rx_within_72',
       'ft_nephrotoxin_vancomycin_rx',
       ...
       'ft_nephrotoxin_ulcer_rx_within_48',
       'ft_nephrotoxin_ulcer_rx_within_72', 'ft_nephrotoxin_propofol_rx',
       'ft_nephrotoxin_propofol_rx_within_24',
       'ft_nephrotoxin_propofol_rx_within_48',
       'ft_nephrotoxin_propofol_rx_within_72', 'ft_nephrotoxin_other_rx',
       'ft_nephrotoxin_other_rx_within_24',
       'ft_nephrotoxin_other_rx_within_48',
       'ft_nephrotoxin_other_rx_within_72'],
      dtype='object', length=101)

,hadm_id,ft_any_nephrotoxin_rx,ft_any_nephrotoxin_rx_within_24,ft_any_nephrotoxin_rx_within_48,ft_any_nephrotoxin_rx_within_72,ft_nephrotoxin_bacitracin_rx,ft_nephrotoxin_bacitracin_rx_within_24,ft_nephrotoxin_bacitracin_rx_within_48,ft_nephrotoxin_bacitracin_rx_within_72,ft_nephrotoxin_vancomycin_rx,ft_nephrotoxin_vancomycin_rx_within_24,ft_nephrotoxin_vancomycin_rx_within_48,ft_nephrotoxin_vancomycin_rx_within_72,ft_nephrotoxin_amphotericin_rx,ft_nephrotoxin_amphotericin_rx_within_24,ft_nephrotoxin_amphotericin_rx_within_48,ft_nephrotoxin_amphotericin_rx_within_72,ft_nephrotoxin_cephalosporin_rx,ft_nephrotoxin_cephalosporin_rx_within_24,ft_nephrotoxin_cephalosporin_rx_within_48,ft_nephrotoxin_cephalosporin_rx_within_72,ft_nephrotoxin_aminoglycoloside_rx,ft_nephrotoxin_aminoglycoloside_rx_within_24,ft_nephrotoxin_aminoglycoloside_rx_within_48,ft_nephrotoxin_aminoglycoloside_rx_within_72,ft_nephrotoxin_ciprofloxacin_rx,ft_nephrotoxin_ciprofloxacin_rx_within_24,ft_nephrotoxin_ciprofloxacin_rx_within_48,ft_nephrotoxin_ciprofloxacin_rx_within_72,ft_nephrotoxin_antibiotics_rx,ft_nephrotoxin_antibiotics_rx_within_24,ft_nephrotoxin_antibiotics_rx_within_48,ft_nephrotoxin_antibiotics_rx_within_72,ft_nephrotoxin_lisinopril_rx,ft_nephrotoxin_lisinopril_rx_within_24,ft_nephrotoxin_lisinopril_rx_within_48,ft_nephrotoxin_lisinopril_rx_within_72,ft_nephrotoxin_ramipril_rx,ft_nephrotoxin_ramipril_rx_within_24,ft_nephrotoxin_ramipril_rx_within_48,ft_nephrotoxin_ramipril_rx_within_72,ft_nephrotoxin_metoprolol_rx,ft_nephrotoxin_metoprolol_rx_within_24,ft_nephrotoxin_metoprolol_rx_within_48,ft_nephrotoxin_metoprolol_rx_within_72,ft_nephrotoxin_candesartan_rx,ft_nephrotoxin_candesartan_rx_within_24,ft_nephrotoxin_candesartan_rx_within_48,ft_nephrotoxin_candesartan_rx_within_72,ft_nephrotoxin_valsartan_rx,ft_nephrotoxin_valsartan_rx_within_24,ft_nephrotoxin_valsartan_rx_within_48,ft_nephrotoxin_valsartan_rx_within_72,ft_nephrotoxin_warfarin_rx,ft_nephrotoxin_warfarin_rx_within_24,ft_nephrotoxin_warfarin_rx_within_48,ft_nephrotoxin_warfarin_rx_within_72,ft_nephrotoxin_blood_pressure_rx,ft_nephrotoxin_blood_pressure_rx_within_24,ft_nephrotoxin_blood_pressure_rx_within_48,ft_nephrotoxin_blood_pressure_rx_within_72,ft_nephrotoxin_furosemide_rx,ft_nephrotoxin_furosemide_rx_within_24,ft_nephrotoxin_furosemide_rx_within_48,ft_nephrotoxin_furosemide_rx_within_72,ft_nephrotoxin_edema_rx,ft_nephrotoxin_edema_rx_within_24,ft_nephrotoxin_edema_rx_within_48,ft_nephrotoxin_edema_rx_within_72,ft_nephrotoxin_ibuprofen_rx,ft_nephrotoxin_ibuprofen_rx_within_24,ft_nephrotoxin_ibuprofen_rx_within_48,ft_nephrotoxin_ibuprofen_rx_within_72,ft_nephrotoxin_naproxen_rx,ft_nephrotoxin_naproxen_rx_within_24,ft_nephrotoxin_naproxen_rx_within_48,ft_nephrotoxin_naproxen_rx_within_72,ft_nephrotoxin_ketoprofen_rx,ft_nephrotoxin_ketoprofen_rx_within_24,ft_nephrotoxin_ketoprofen_rx_within_48,ft_nephrotoxin_ketoprofen_rx_within_72,ft_nephrotoxin_nsaid_rx,ft_nephrotoxin_nsaid_rx_within_24,ft_nephrotoxin_nsaid_rx_within_48,ft_nephrotoxin_nsaid_rx_within_72,ft_nephrotoxin_cimetidine_rx,ft_nephrotoxin_cimetidine_rx_within_24,ft_nephrotoxin_cimetidine_rx_within_48,ft_nephrotoxin_cimetidine_rx_within_72,ft_nephrotoxin_ulcer_rx,ft_nephrotoxin_ulcer_rx_within_24,ft_nephrotoxin_ulcer_rx_within_48,ft_nephrotoxin_ulcer_rx_within_72,ft_nephrotoxin_propofol_rx,ft_nephrotoxin_propofol_rx_within_24,ft_nephrotoxin_propofol_rx_within_48,ft_nephrotoxin_propofol_rx_within_72,ft_nephrotoxin_other_rx,ft_nephrotoxin_other_rx_within_24,ft_nephrotoxin_other_rx_within_48,ft_nephrotoxin_other_rx_within_72
10123,120034,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41507,182720,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [7]:
nephrotoxin_features.describe().T.sort_values('mean', ascending=False).head()

,count,mean,std,min,25%,50%,75%,max
ft_any_nephrotoxin_rx,50216.0,0.828959,0.376549,0.0,1.0,1.0,1.0,1.0
ft_any_nephrotoxin_rx_within_72,50216.0,0.785546,0.410447,0.0,1.0,1.0,1.0,1.0
ft_any_nephrotoxin_rx_within_48,50216.0,0.761968,0.425883,0.0,1.0,1.0,1.0,1.0
ft_any_nephrotoxin_rx_within_24,50216.0,0.715230,0.451309,0.0,0.0,1.0,1.0,1.0
ft_nephrotoxin_blood_pressure_rx,50216.0,0.619842,0.485430,0.0,0.0,1.0,1.0,1.0


**Contrast Imaging Procedures**
---
Radiology/Imaging that uses contrast dyes might cause AKI in patients with chronic kidney disease or those who are particular risks. In these settings, the dye itself is a nephrotoxin. **This feature is currently broken because I'm using some outdated CPT codes**.

In [8]:
cptevents = read_prod_data('cptevents')
contrast_imaging_feature = create_contrast_imaging_feature(cptevents)
printer('contrast dyes features')
contrast_imaging_feature.shape
contrast_imaging_feature.sample(5)
contrast_imaging_feature.describe()

cptevents (573146, 12)
contrast dyes features
-------------------


(44148, 2)

,hadm_id,ft_contrast_imaging
41813,194640,0
19399,144029,0
14305,132392,0
3065,106945,0
1205,102752,0


,ft_contrast_imaging
count,44148.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


**Mechanical Ventilation**
---
Mechanical Ventilation according to some of the papers I read was a risk for kidney injury.

In [9]:
mechanical_ventilation_feature = create_mechanical_ventilation_feature(cptevents)
del cptevents
printer('mechanical ventilation features')
mechanical_ventilation_feature.shape
mechanical_ventilation_feature.sample(5)
mechanical_ventilation_feature.describe()

mechanical ventilation features
-------------------


(44148, 2)

,hadm_id,ft_mechanical_ventilation
42406,195983,0
18484,141934,0
36860,183584,0
25101,157067,1
8555,119239,1


,ft_mechanical_ventilation
count,44148.000000
mean,0.371591
std,0.483235
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


**Charts Data**
---
View the feature space that's created for the chart data. Each feature space looks at another data point in the charts table.

In [10]:
# one_bin = False

In [11]:
# d_items = read_prod_data('d_items')

# if not one_bin:
#     bins = [hex(i)[2] + c for i in range(0, 16) for c in [hex(d)[2] for d in range(0, 16)]]   
# else:
#     bins = ['00', 'ff', '22', '3e']

# # chart features
# chart_features = [charts_data_wrapper(b, 
#                                       d_items, 
#                                       df, 
#                                       demographic_features,
#                                      i) for i,b in enumerate(bins)]
# chart_features = pd.concat(chart_features, sort=False)

# del d_items
# printer('charts features')
# chart_features.shape
# chart_features.columns
# chart_features.sample(5)

In [12]:
# chart_features.to_csv(result_dir + 'chart_features.csv', index=False)

In [13]:
chart_features = pd.read_csv(result_dir + 'chart_features.csv', dtype={'hadm_id': str})
printer('chart data features')
chart_features.shape
chart_features.columns
chart_features.sample(5)
printer('\n')
chart_features.describe().T.sort_values('mean', ascending=False)

FileNotFoundError: [Errno 2] File b'results/2019-12-02-17-46/chart_features.csv' does not exist: b'results/2019-12-02-17-46/chart_features.csv'

**Merge & Create Final Dataframe**
---

In [ ]:
features = [
    df.drop(['subject_id', 'admittime', 'dischtime'], axis=1),
    demographic_features,
    chart_features,
    diabetes_hcc_feature,
    ckd4_hcc_feature,
    ckd5_hcc_feature,
    chf_hcc_feature,
    vascular_disease_hcc_feature,
    prior_admission_features,
    nephrotoxin_features,
    contrast_imaging_feature,
    mechanical_ventilation_feature
]

data = merge_features(features)
printer('final dataframe')
print(data.shape)

In [ ]:
data.describe()

In [ ]:
data.to_csv(result_dir + 'all_features.csv', index=False)